In [1]:
import pandas as pd
import sys
sys.path.append("../../hsic_codes/") 
from mlp_pd import *

print('before load data')
path = "../../download_scripts/lung/"


label_key = 'cell_ontology_class'

data = pd.read_csv(path+ 'facs_train.csv', index_col='index',)
test_data = pd.read_csv(path+'facs_test.csv', index_col='index')

ood_data = pd.read_csv(path+ 'droplet_train.csv', index_col='index',)
ood_test_data = pd.read_csv(path+'droplet_test.csv', index_col='index')
ood = pd.concat([ood_data, ood_test_data])
del ood_data, ood_test_data

before load data


In [2]:
# torch.cuda.device(1)
# torch.cuda.set_device(1)
torch.cuda.current_device()

0

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')


s_x, s_y = 1, 2

num_classes = np.unique(data[label_key]).shape[0]
input_dim = data.shape[1]-1


Using device: cuda
GeForce GTX TITAN X
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:

model_hsic = MLP(input_dim, num_classes=num_classes, layer_sizes=[1024, 128, 128], dr_rate=0.075)
model_hsic.to(device)





MLP(
  (FC): Sequential(
    (L0): Linear(in_features=4000, out_features=1024, bias=False)
    (B0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A0): ReLU()
    (D0): Dropout(p=0.075)
    (L1): Linear(in_features=1024, out_features=128, bias=False)
    (B1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A1): ReLU()
    (D1): Dropout(p=0.075)
    (L2): Linear(in_features=128, out_features=128, bias=False)
    (B2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A2): ReLU()
    (D2): Dropout(p=0.075)
    (output): Linear(in_features=128, out_features=32, bias=True)
  )
)

In [5]:
trainer_hsic = Trainer(model_hsic, data, label_key=label_key, s_x=s_x, s_y =s_y)
trainer_hsic.train_HSIC( 150, 32, weight_decay=0)




Epoch 00/150 Batch 0110/110, Loss:    0.0050
Epoch 01/150 Batch 0110/110, Loss:    0.0013
Epoch 02/150 Batch 0110/110, Loss:    0.0002
Epoch 03/150 Batch 0110/110, Loss:    0.0006
Epoch 04/150 Batch 0110/110, Loss:    0.0003
Epoch 05/150 Batch 0110/110, Loss:    0.0005
Epoch 05, Loss_valid:    0.0300, acc_valid:    0.9442
Epoch 06/150 Batch 0110/110, Loss:    0.0001
Epoch 07/150 Batch 0110/110, Loss:    0.0001
Epoch 08/150 Batch 0110/110, Loss:    0.0001
Epoch 09/150 Batch 0110/110, Loss:    0.0001
Epoch 10/150 Batch 0110/110, Loss:    0.0001
Epoch 10, Loss_valid:    0.0285, acc_valid:    0.9426
Epoch 11/150 Batch 0110/110, Loss:    0.0001
Epoch 12/150 Batch 0110/110, Loss:    0.0000
Epoch 13/150 Batch 0110/110, Loss:    0.0001
Epoch 14/150 Batch 0110/110, Loss:    0.0001
Epoch 15/150 Batch 0110/110, Loss:    0.0001
Epoch 15, Loss_valid:    0.0301, acc_valid:    0.9410
Epoch 16/150 Batch 0110/110, Loss:    0.0001
Epoch 17/150 Batch 0110/110, Loss:    0.0001
Epoch 18/150 Batch 0110/110,

In [7]:
model_hsic.double()

MLP(
  (FC): Sequential(
    (L0): Linear(in_features=4000, out_features=1024, bias=False)
    (B0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A0): ReLU()
    (D0): Dropout(p=0.075)
    (L1): Linear(in_features=1024, out_features=128, bias=False)
    (B1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A1): ReLU()
    (D1): Dropout(p=0.075)
    (L2): Linear(in_features=128, out_features=128, bias=False)
    (B2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A2): ReLU()
    (D2): Dropout(p=0.075)
    (output): Linear(in_features=128, out_features=32, bias=True)
  )
)

In [8]:
#train report
from sklearn.metrics import classification_report

print("******************")
print("train on facs")
print("s_x, s_y:", s_x, s_y)
x = trainer_hsic.train_data.drop(columns=[label_key]).values
y_pred = model_hsic.label_encoder.inverse_transform(model_hsic.predict(x))
print(classification_report(np.array(trainer_hsic.train_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("validation")
x = trainer_hsic.validation_data.drop(columns=[label_key]).values
y_pred = model_hsic.label_encoder.inverse_transform(model_hsic.predict(x))
print(classification_report(np.array(trainer_hsic.validation_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("facs test")
x = test_data.drop(columns=[label_key]).values
y_pred = model_hsic.label_encoder.inverse_transform(model_hsic.predict(x))
print(classification_report(np.array(test_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("droplet (ood1)")
x = ood.drop(columns=[label_key]).values
y_pred = model_hsic.label_encoder.inverse_transform(model_hsic.predict(x))
print(classification_report(np.array(ood[label_key]).reshape(-1,1), y_pred))

******************
train on facs
s_x, s_y: 1 2
                                                 precision    recall  f1-score   support

                                         B cell       1.00      1.00      1.00       179
                CD4-positive, alpha-beta T cell       1.00      1.00      1.00        42
                CD8-positive, alpha-beta T cell       1.00      1.00      1.00        80
                                        NK cell       1.00      1.00      1.00        37
                                         T cell       1.00      1.00      1.00        18
                               adventitial cell       1.00      1.00      1.00       196
                   bronchial smooth muscle cell       1.00      1.00      1.00      1422
ciliated columnar cell of tracheobronchial tree       1.00      1.00      1.00        39
                             classical monocyte       1.00      1.00      1.00       139
                        club cell of bronchiole       1.00    

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                                 precision    recall  f1-score   support

                                         B cell       0.98      0.93      0.95        56
                CD4-positive, alpha-beta T cell       0.70      0.78      0.74         9
                CD8-positive, alpha-beta T cell       0.79      0.70      0.75        27
                                        NK cell       1.00      0.86      0.92         7
                                         T cell       1.00      0.80      0.89         5
                               adventitial cell       0.98      0.93      0.96        58
                   bronchial smooth muscle cell       0.94      0.98      0.96       428
ciliated columnar cell of tracheobronchial tree       1.00      1.00      1.00         8
                             classical monocyte       0.93      1.00      0.96        41
                        club cell of bronchiole       0.57      0.80      0.67         5
                    

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                                                 precision    recall  f1-score   support

                                         B cell       0.99      0.99      0.99      1501
                CD4-positive, alpha-beta T cell       0.47      0.49      0.48       551
                CD8-positive, alpha-beta T cell       0.44      0.47      0.45       870
                                        NK cell       0.99      0.90      0.94      1193
                                         T cell       0.96      0.54      0.69       251
                               adventitial cell       0.96      0.86      0.91       526
                            alveolar macrophage       0.00      0.00      0.00      1366
                                       basophil       0.00      0.00      0.00       130
                   bronchial smooth muscle cell       0.65      0.99      0.79      2339
ciliated columnar cell of tracheobronchial tree       0.98      1.00      0.99        57
                    

### nll results

In [9]:

model_nll = MLP(input_dim, num_classes=num_classes, layer_sizes=[512, 128, 64], dr_rate=0.05)
model_nll.to(device)
trainer_nll = Trainer(model_nll, data, label_key=label_key, s_x=s_x, s_y =s_y)
trainer_nll.train_nll( 150, 32, weight_decay=0.0001)




Epoch 00/150 Batch 0110/110, Loss:    1.0516
Epoch 01/150 Batch 0110/110, Loss:    0.0922
Epoch 02/150 Batch 0110/110, Loss:    0.0882
Epoch 03/150 Batch 0110/110, Loss:    0.0154
Epoch 04/150 Batch 0110/110, Loss:    0.0042
Epoch 05/150 Batch 0110/110, Loss:    0.0005
Epoch 05, Loss_valid:    4.8954, acc_valid:    0.9506
Epoch 06/150 Batch 0110/110, Loss:    0.0009
Epoch 07/150 Batch 0110/110, Loss:    0.0002
Epoch 08/150 Batch 0110/110, Loss:    0.0001
Epoch 09/150 Batch 0110/110, Loss:    0.0003
Epoch 10/150 Batch 0110/110, Loss:    0.0025
Epoch 10, Loss_valid:    8.2911, acc_valid:    0.8963
Epoch 11/150 Batch 0110/110, Loss:    0.0268
Epoch 12/150 Batch 0110/110, Loss:    0.0063
Epoch 13/150 Batch 0110/110, Loss:    0.0547
Epoch 14/150 Batch 0110/110, Loss:    0.0089
Epoch 15/150 Batch 0110/110, Loss:    0.0006
Epoch 15, Loss_valid:    8.5607, acc_valid:    0.9410
Epoch 16/150 Batch 0110/110, Loss:    0.0003
Epoch 17/150 Batch 0110/110, Loss:    0.0002
Epoch 18/150 Batch 0110/110,

In [10]:
model_nll.double()

MLP(
  (FC): Sequential(
    (L0): Linear(in_features=4000, out_features=512, bias=False)
    (B0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A0): ReLU()
    (D0): Dropout(p=0.05)
    (L1): Linear(in_features=512, out_features=128, bias=False)
    (B1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A1): ReLU()
    (D1): Dropout(p=0.05)
    (L2): Linear(in_features=128, out_features=64, bias=False)
    (B2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A2): ReLU()
    (D2): Dropout(p=0.05)
    (output): Linear(in_features=64, out_features=32, bias=True)
  )
)

In [11]:
#train report
from sklearn.metrics import classification_report

print("******************")
print("train on facs")
print("s_x, s_y:", s_x, s_y)
x = trainer_nll.train_data.drop(columns=[label_key]).values
y_pred = model_nll.label_encoder.inverse_transform(model_nll.predict(x))
print(classification_report(np.array(trainer_nll.train_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("validation")
x = trainer_nll.validation_data.drop(columns=[label_key]).values
y_pred = model_nll.label_encoder.inverse_transform(model_nll.predict(x))
print(classification_report(np.array(trainer_nll.validation_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("facs test")
x = test_data.drop(columns=[label_key]).values
y_pred = model_nll.label_encoder.inverse_transform(model_nll.predict(x))
print(classification_report(np.array(test_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("droplet (ood1)")
x = ood.drop(columns=[label_key]).values
y_pred = model_nll.label_encoder.inverse_transform(model_nll.predict(x))
print(classification_report(np.array(ood[label_key]).reshape(-1,1), y_pred))

******************
train on facs
s_x, s_y: 1 2
                                                 precision    recall  f1-score   support

                                         B cell       1.00      1.00      1.00       179
                CD4-positive, alpha-beta T cell       1.00      1.00      1.00        42
                CD8-positive, alpha-beta T cell       1.00      1.00      1.00        80
                                        NK cell       1.00      1.00      1.00        37
                                         T cell       1.00      1.00      1.00        18
                               adventitial cell       1.00      1.00      1.00       196
                   bronchial smooth muscle cell       1.00      1.00      1.00      1422
ciliated columnar cell of tracheobronchial tree       1.00      1.00      1.00        39
                             classical monocyte       1.00      1.00      1.00       139
                        club cell of bronchiole       1.00    

                                                 precision    recall  f1-score   support

                                         B cell       0.90      0.99      0.95      1501
                CD4-positive, alpha-beta T cell       0.56      0.49      0.52       551
                CD8-positive, alpha-beta T cell       0.43      0.75      0.54       870
                                        NK cell       0.98      0.89      0.93      1193
                                         T cell       0.91      0.47      0.62       251
                               adventitial cell       0.91      0.89      0.90       526
                            alveolar macrophage       0.00      0.00      0.00      1366
                                       basophil       0.00      0.00      0.00       130
                   bronchial smooth muscle cell       0.60      0.99      0.75      2339
ciliated columnar cell of tracheobronchial tree       0.98      0.96      0.97        57
                    

### TRADES

In [20]:
model_trades = MLP(input_dim, num_classes=num_classes, layer_sizes=[512, 128, 64], dr_rate=0.05)
model_trades.to(device)
trainer_trades = Trainer(model_trades, data, label_key=label_key,)
trainer_trades.train_trades( n_epochs = 150, batch_size = 32)



In [21]:
le = model_trades.label_encoder

In [22]:
from mlp_pd import label_encoder
c,_ =label_encoder(data, le, label_key=label_key)

In [24]:
model_trades.double()

MLP(
  (FC): Sequential(
    (L0): Linear(in_features=4000, out_features=512, bias=False)
    (B0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A0): ReLU()
    (D0): Dropout(p=0.05)
    (L1): Linear(in_features=512, out_features=128, bias=False)
    (B1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A1): ReLU()
    (D1): Dropout(p=0.05)
    (L2): Linear(in_features=128, out_features=64, bias=False)
    (B2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (A2): ReLU()
    (D2): Dropout(p=0.05)
    (output): Linear(in_features=64, out_features=32, bias=True)
  )
)

In [25]:
#train report
from sklearn.metrics import classification_report

print("******************")
print("train on facs")
print("s_x, s_y:", s_x, s_y)
x = trainer_trades.train_data.drop(columns=[label_key]).values
y_pred = model_trades.label_encoder.inverse_transform(model_trades.predict(x))
print(classification_report(np.array(trainer_trades.train_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("validation")
x = trainer_trades.validation_data.drop(columns=[label_key]).values
y_pred = model_trades.label_encoder.inverse_transform(model_trades.predict(x))
print(classification_report(np.array(trainer_trades.validation_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("facs test")
x = test_data.drop(columns=[label_key]).values
y_pred = model_trades.label_encoder.inverse_transform(model_trades.predict(x))
print(classification_report(np.array(test_data[label_key]).reshape(-1,1), y_pred))

print("******************")
print("droplet (ood1)")
x = ood.drop(columns=[label_key]).values
y_pred = model_trades.label_encoder.inverse_transform(model_trades.predict(x))
print(classification_report(np.array(ood[label_key]).reshape(-1,1), y_pred))

******************
train on facs
s_x, s_y: 1 2
                                                 precision    recall  f1-score   support

                                         B cell       0.99      1.00      1.00       179
                CD4-positive, alpha-beta T cell       0.95      1.00      0.98        42
                CD8-positive, alpha-beta T cell       1.00      0.97      0.99        80
                                        NK cell       1.00      1.00      1.00        37
                                         T cell       0.72      1.00      0.84        18
                               adventitial cell       1.00      1.00      1.00       196
                   bronchial smooth muscle cell       1.00      1.00      1.00      1422
ciliated columnar cell of tracheobronchial tree       1.00      1.00      1.00        39
                             classical monocyte       1.00      1.00      1.00       139
                        club cell of bronchiole       0.90    

                                                 precision    recall  f1-score   support

                                         B cell       0.95      0.99      0.97      1501
                CD4-positive, alpha-beta T cell       0.48      0.57      0.52       551
                CD8-positive, alpha-beta T cell       0.45      0.52      0.49       870
                                        NK cell       0.99      0.92      0.95      1193
                                         T cell       0.95      0.51      0.66       251
                               adventitial cell       0.81      0.90      0.86       526
                            alveolar macrophage       0.00      0.00      0.00      1366
                                       basophil       0.00      0.00      0.00       130
                   bronchial smooth muscle cell       0.22      0.97      0.36      2339
ciliated columnar cell of tracheobronchial tree       0.97      1.00      0.98        57
                    